In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import timeit

from torch.autograd import Variable

import numpy as np
import matplotlib.pyplot as plt
import sklearn

%matplotlib inline
%load_ext autoreload
%autoreload 2

from sklearn.preprocessing import OneHotEncoder
from uppercase import Dataset

/Users/darth/.miniconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
window_size = 5
alphabet_size = 100

train = Dataset("data/uppercase_data_train.txt", window_size, alphabet=alphabet_size)
dev = Dataset("data/uppercase_data_dev.txt", window_size, alphabet=train.alphabet)
test = Dataset("data/uppercase_data_test.txt", window_size, alphabet=train.alphabet)

In [76]:
X, y = test.all_data()
X_test, y_test = X[:, :], y[:,]
X_test = OneHotEncoder(alphabet_size, sparse=False).fit_transform(X_test)

In [74]:
import visdom
vis = visdom.Visdom()

def torch_batch(batch, onehot=False):
    X, y = batch
    if onehot:
        X = OneHotEncoder(alphabet_size, sparse=False).fit_transform(X)
        
    X = Variable(torch.from_numpy(X).float())
    y = Variable(torch.from_numpy(y.astype(np.long)))
    
    return X, y

def accuracy(scores, y):
    return (scores.round().long() == y).float().mean().data[0]

In [75]:
class Ravel(nn.Module):
    def forward(self, x):
        return x.view(-1)

model = nn.Sequential(
    nn.Linear(alphabet_size*(2*window_size+1), 200),
    nn.ReLU(),
    nn.Linear(200, 200),
    nn.ReLU(),
    nn.Linear(200, 200),
    nn.ReLU(),
    nn.Linear(200, 200),
    nn.ReLU(),
    nn.Linear(200, 1),
    nn.Sigmoid(),
    Ravel(),
)

optimizer = optim.Adam(model.parameters())
loss = nn.BCELoss()

In [77]:
losses = []
accuracies = []

dev_losses = []
dev_accuracies = []

i = 0
for _ in range(2):
    while not train.epoch_finished():
        i += 1
        X, y = torch_batch(train.next_batch(20), onehot=True)
        
        optimizer.zero_grad()
        scores = model(X)
        
        output = loss(scores, y.float())
        output.backward()
        optimizer.step()
        
        if i % 5 == 0:
            losses.append(output.data[0])
            accuracies.append(accuracy(scores, y))

            vis.line(np.array(losses), win="loss", opts={"title": "train/loss"})
            vis.line(np.array(accuracies), win="acc", opts={"title": "train/acc"})

        if i % 200 == 0:
            idx = np.random.choice(y_test.shape[0], 5000)
            X, y = torch_batch((X_test[idx,:], y_test[idx]))
            
            scores = model(X)
            output = loss(scores, y.float())
        
            dev_losses.append(output.data[0])
            dev_accuracies.append(accuracy(scores, y))
            vis.line(np.array(dev_losses), win="dev/loss", opts={"title": "dev/loss"})
            vis.line(np.array(dev_accuracies), win="dev/acc", opts={"title": "dev/acc"})


KeyboardInterrupt: 

In [32]:
%debug

> <ipython-input-22-e916ae25e19d>(10)torch_batch()
      8 
      9     X = Variable(torch.from_numpy(X).float())
---> 10     y = Variable(torch.from_numpy(y.astype(np.long)))
     11 
     12     return X, y

ipdb> y
False
ipdb> idx
998
ipdb> np.random.choice(1000, 100)
array([315, 820, 791, 849, 972, 572, 777, 364, 435, 916, 327, 969, 254,
       706,  85, 198, 613, 126, 777, 200, 144, 445, 457,  73, 322, 902,
       658,  57, 441, 157, 436, 138, 869, 226, 884, 653, 936, 279, 643,
       800, 272, 119, 998, 903, 972, 606, 613, 782, 929, 681, 460, 627,
       344, 642, 812, 786,  85, 365, 485, 882,  83, 490, 701, 691, 825,
       624, 874, 517, 588, 196, 454,  14, 876, 427, 419, 469, 378, 950,
       447, 875, 417, 268, 523,  67, 970, 280, 951,  15, 794, 836, 996,
       448, 739, 327, 706, 205, 655, 234, 679, 761])
ipdb> exit


In [9]:
import visdom
import numpy as np

In [19]:
data = np.array([[1], [2]])

In [17]:
res = vis.line(data)

# trace = dict(x=[1, 2, 3], y=[4, 5, 6]             
# layout = dict(title="First Plot", xaxis={'title': 'x1'}, yaxis={'title': 'x2'})

# vis._send({'data': [trace], 'layout': layout, 'win': 'mywin'})

'window_3609aecbf63c90'

In [39]:
vis.line(np.array([2,4,2,3,4,3,3,2]), win="gege")

'gege'